Cor Vermelha

Banco de dados SISMULTAS - 2013 à 2022
Juntar todas as bases em uma única base para podermos visualizar as informações referêntes a cada uma delas

In [ ]:
!pip install seaborn --quiet
!npm install localtunnel --quiet
!pip install streamlit --quiet
!pip install plotly --quiet
!pip install geopandas --quiet
!apt-get install -y libspatialindex-dev --quiet
!pip install rtree --quiet


⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏
added 22 packages in 7s
⠏
⠏3 packages are looking for funding
⠏  run `npm fund` for details
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 73.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 133.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 8.8 MB/s eta 0:00:00
Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  libspatialindex-c6 libspatialindex6
The following NEW packages will be installed:
  libspatialindex-c6 libspatialindex-dev libspatialindex6
0 upgraded, 3 newly installed, 0 to remove and 35 not upgraded.
Need to get 319 kB of archives.
After this operation, 1,416 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libspatialindex6 amd64 1.9.3-2 [

In [ ]:
%%writefile app.py

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import streamlit as st
import os
# Importações para mapas
import plotly.express as px
import geopandas as gpd

# --- Configuração da Página ---
st.set_page_config(
    page_title="Análise de Infrações de Trânsito",
    page_icon="🗺️",
    layout="wide"
)

# --- Funções de Apoio e Cache ---

@st.cache_data # Otimização: armazena o resultado da função em cache
def carregar_dados(lista_de_arquivos):
    """Lê, concatena e pré-processa todos os arquivos CSV enviados."""
    all_dfs = []
    for uploaded_file in lista_de_arquivos:
        try:
            year_str = uploaded_file.name.split('_')[-1].split('.')[0]
            year = int(year_str)
            df = pd.read_csv(uploaded_file, sep=";", encoding="latin1")
            df['Ano_Referencia'] = year
            all_dfs.append(df)
        except Exception as e:
            st.error(f"Erro ao processar o arquivo {uploaded_file.name}: {e}")
            return pd.DataFrame() # Retorna dataframe vazio em caso de erro

    if not all_dfs:
        return pd.DataFrame()

    # Concatena e limpa os dados
    combined_df = pd.concat(all_dfs, ignore_index=True)

    # Seleciona e renomeia colunas para manter um padrão limpo
    colunas_necessarias = {
        'escopo_autuacao': 'escopo_autuacao',
        'mes': 'mes',
        'uf': 'uf',
        'quantidade_autos': 'quantidade_autos',
        'codigo_infracao': 'codigo_infracao',
        'amparo_legal': 'amparo_legal',
        'descricao_infracao': 'descricao_infracao',
        'Ano_Referencia': 'Ano_Referencia'
    }
    # Mantém apenas as colunas que existem no dataframe
    colunas_existentes = [col for col in colunas_necessarias if col in combined_df.columns]
    combined_df = combined_df[colunas_existentes]

    combined_df['quantidade_autos'] = pd.to_numeric(combined_df['quantidade_autos'], errors='coerce')
    combined_df.dropna(subset=['quantidade_autos'], inplace=True)
    combined_df['uf'] = combined_df['uf'].str.upper()
    combined_df['mes'] = combined_df['mes'].str.lower()

    return combined_df

@st.cache_data
def convert_df_to_csv(df):
    """Converte o DataFrame para um arquivo CSV em memória, pronto para download."""
    return df.to_csv(index=False).encode('utf-8')

# --- Barra Lateral (Sidebar) ---

st.sidebar.image("https://anttitajai.com.br/wp-content/uploads/2024/05/antt.webp", width=100)
st.sidebar.title("Filtros da Análise")

uploaded_files = st.sidebar.file_uploader(
    "Escolha os arquivos CSV (2013 a 2022)",
    type="csv",
    accept_multiple_files=True
)

# --- Lógica Principal ---

if not uploaded_files:
    st.title("Bem-vindo à Análise de Infrações de Trânsito 🚗")
    st.info("Por favor, faça o upload dos arquivos CSV na barra lateral para começar.")
    st.stop() # Interrompe a execução se não houver arquivos

# Carrega os dados usando a função com cache
df_principal = carregar_dados(uploaded_files)

if df_principal.empty:
    st.error("Nenhum dado válido foi carregado. Verifique os arquivos.")
    st.stop()

# --- Filtros Dinâmicos na Sidebar ---

# Slider de Anos
min_year = int(df_principal['Ano_Referencia'].min())
max_year = int(df_principal['Ano_Referencia'].max())
year_range = st.sidebar.slider(
    'Selecione o Período de Anos',
    min_year,
    max_year,
    (min_year, max_year)
)

# Filtro de UF (Estado)
ufs_disponiveis = sorted(df_principal['uf'].unique())
uf_selecionada = st.sidebar.multiselect(
    'Selecione a UF (opcional)',
    options=ufs_disponiveis,
    default=[] # Por padrão, nenhuma UF é selecionada
)

# --- Aplicação dos Filtros ---

# Cria o dataframe filtrado que será usado em todas as análises
df_filtrado = df_principal[
    (df_principal['Ano_Referencia'] >= year_range[0]) &
    (df_principal['Ano_Referencia'] <= year_range[1])
]

if uf_selecionada:
    df_filtrado = df_filtrado[df_filtrado['uf'].isin(uf_selecionada)]

if df_filtrado.empty:
    st.warning("Nenhum dado encontrado para os filtros selecionados.")
    st.stop()

# --- Interface com Abas ---

st.title("Análise de Infrações de Trânsito no Brasil")
st.write(f"Dados para o período de **{year_range[0]}** a **{year_range[1]}**.")
if uf_selecionada:
    st.write(f"Filtrado para as UFs: **{', '.join(uf_selecionada)}**")


tab1, tab2,tab3, tab4, tab5 = st.tabs([
    "📊 Análise Geral",
    "📜 Análise de Infrações",
    "📈 Análise Temporal e Gráfica",
    "🗺️ Análise Geográfica",
    "📥 Exportação"
])

# --- Aba 1: Análise Geral ---
with tab1:
    st.header("Métricas Chave (KPIs)")

    # Lógica para obter os anos de análise e o ano anterior para comparação
    ano_final = year_range[1]
    ano_inicial_selecionado = year_range[0]
    ano_anterior = ano_final - 1

    # Filtrar dados para o ano final e o ano anterior (usando df_principal para ter todos os dados históricos)
    df_ano_final = df_principal[df_principal['Ano_Referencia'] == ano_final]
    df_ano_anterior = df_principal[df_principal['Ano_Referencia'] == ano_anterior]

    # Calcular totais de autos para o ano final e o ano anterior
    total_ano_final = df_ano_final['quantidade_autos'].sum()
    total_ano_anterior = df_ano_anterior['quantidade_autos'].sum()

    # Calcular o total de autos no período selecionado (já feito com df_filtrado)
    total_autos_periodo_selecionado = int(df_filtrado['quantidade_autos'].sum())

    # Encontrar a UF e Infração com mais autos no período selecionado
    top_uf_periodo = df_filtrado.groupby('uf')['quantidade_autos'].sum().idxmax()
    top_infracao_desc_periodo = df_filtrado.groupby('descricao_infracao')['quantidade_autos'].sum().idxmax()

    # Calcular variação percentual
    delta_percentual = 0
    if total_ano_anterior > 0:
        delta_percentual = ((total_ano_final - total_ano_anterior) / total_ano_anterior) * 100

    col1, col2, col3, col4 = st.columns(4)

    # KPI: Total de Autos no Período Selecionado
    col1.metric("Total de Autos no Período", f"{total_autos_periodo_selecionado:,}".replace(",", "."))

    # KPI: Total de Autos no Ano Anterior (antes da variação)
    col2.metric(f"Total de Autos em {ano_anterior}", f"{int(total_ano_anterior):,}".replace(",", ".")) # Mostrar o valor absoluto do ano anterior

    # KPI: Variação Percentual vs Ano Anterior
    col3.metric(f"Variação vs Ano Anterior", f"{delta_percentual:.2f}%")

    # KPI: UF com Mais Autos no Período Selecionado
    col4.metric(f"UF com Mais Autos no Período", top_uf_periodo)

    st.markdown("---")

# --- Aba 2: Análise de Infrações ---
with tab2:
    st.header("Análise por Tipo de Infração")
    st.info("Aqui exploramos quais as infrações mais comuns com base na descrição fornecida nos dados.")

    top_n = st.slider("Selecione o número de infrações para exibir no gráfico", 5, 20, 10)

    # Gráfico de barras: Top N infrações
    infracoes_comuns = df_filtrado.groupby('descricao_infracao')['quantidade_autos'].sum().nlargest(top_n).sort_values(ascending=True)
    fig_infracoes = px.bar(
        infracoes_comuns,
        x='quantidade_autos',
        y=infracoes_comuns.index,
        orientation='h',
        title=f'Top {top_n} Infrações Mais Comuns por Descrição'
    )
    fig_infracoes.update_layout(yaxis_title="Descrição da Infração", xaxis_title="Quantidade de Autos")
    st.plotly_chart(fig_infracoes, use_container_width=True)

    with st.expander("Ver tabela de Amparo Legal das infrações mais comuns"):
        st.dataframe(df_filtrado[df_filtrado['descricao_infracao'].isin(infracoes_comuns.index)][['descricao_infracao', 'amparo_legal']].drop_duplicates())

# --- Aba 3: Análise Temporal e Gráfica ---

with tab3:
    st.header("Análise Gráfica de Infrações")
    st.info("Visualização das tendências temporais e distribuições por estado e tipo de infração.")

    # Gráfico de Linhas: Quantidade de Autos por Ano
    st.subheader("Total de Autos por Ano")
    autos_por_ano_temp = df_filtrado.groupby('Ano_Referencia')['quantidade_autos'].sum().reset_index()
    fig_line_anos = px.line(autos_por_ano_temp, x='Ano_Referencia', y='quantidade_autos', title='Quantidade de Autos por Ano', markers=True)
    fig_line_anos.update_layout(xaxis_title="Ano", yaxis_title="Quantidade de Autos")
    st.plotly_chart(fig_line_anos, use_container_width=True)

    st.markdown("---")

    # Gráfico de Barras: Autos por Estado
    st.subheader("Total de Autos por Estado (UF)")
    autos_por_uf_temp = df_filtrado.groupby('uf')['quantidade_autos'].sum().reset_index().sort_values(by='quantidade_autos', ascending=False)
    fig_bar_uf = px.bar(
        autos_por_uf_temp,
        x='uf',
        y='quantidade_autos',
        title='Quantidade de Autos por Estado (UF)',
        text='quantidade_autos' # Adiciona rótulos de texto com os valores nas barras
    )
    fig_bar_uf.update_layout(xaxis_title="Estado (UF)", yaxis_title="Quantidade de Autos")
    # Ajusta o formato do texto nas barras para incluir separador de milhares se necessário
    fig_bar_uf.update_traces(texttemplate='%{text:,s}'.replace(',', '.'), textposition='outside')
    st.plotly_chart(fig_bar_uf, use_container_width=True)

    st.markdown("---")


# --- Aba 4: Análise Geográfica ---
with tab4:
    st.header("Distribuição Geográfica das Infrações")

    # Agregar dados por estado (UF) para o período selecionado
    autos_por_uf = df_filtrado.groupby('uf')['quantidade_autos'].sum().reset_index()
    autos_por_uf.rename(columns={'uf': 'sigla', 'quantidade_autos': 'Total_Autos'}, inplace=True)

    try:
        geojson_url = 'https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/brazil-states.geojson'
        brasil_gdf = gpd.read_file(geojson_url)

        # Mesclar dados de autos com o GeoDataFrame
        brasil_gdf = brasil_gdf.merge(autos_por_uf, left_on='sigla', right_on='sigla', how='left')
        brasil_gdf['Total_Autos'] = brasil_gdf['Total_Autos'].fillna(0) # Preencher estados sem dados com 0

        # Mapa Geral de Autos por Estado
        fig_mapa_geral = px.choropleth(
            brasil_gdf, geojson=brasil_gdf.geometry, locations=brasil_gdf.index,
            color='Total_Autos',
            hover_name='sigla', hover_data={'Total_Autos': True},
            color_continuous_scale="Reds", # Alterado para escala de branco a vermelho
            projection='mercator',
            title=f'Distribuição Geral de Autos por Estado ({year_range[0]}-{year_range[1]})'
        )
        fig_mapa_geral.update_geos(fitbounds="locations", visible=False)
        fig_mapa_geral.update_layout(margin={"r":0, "t":40, "l":0, "b":0})
        st.plotly_chart(fig_mapa_geral, use_container_width=True)

        st.markdown("---")

        st.subheader("Top 5 Estados com mais Autos")

        # Destacar Top 5 com Mais Autos
        top5_mais_siglas = autos_por_uf.nlargest(5, 'Total_Autos')['sigla'].tolist()
        brasil_gdf['destaque_mais'] = brasil_gdf['sigla'].apply(
            lambda sigla: 'Top 5 Mais' if sigla in top5_mais_siglas else 'Outros'
        )

        fig_mapa_mais = px.choropleth(
            brasil_gdf, geojson=brasil_gdf.geometry, locations=brasil_gdf.index,
            color='destaque_mais',
            hover_name='sigla', hover_data={'Total_Autos': True},
            projection='mercator',
            color_discrete_map={'Top 5 Mais': 'red', 'Outros': 'lightgrey'}, # Cor vermelha para mais autos
            title=f'Destaque dos 5 Estados com Mais Autos ({year_range[0]}-{year_range[1]})'
        )
        fig_mapa_mais.update_geos(fitbounds="locations", visible=False)
        fig_mapa_mais.update_layout(margin={"r":0, "t":40, "l":0, "b":0})
        st.plotly_chart(fig_mapa_mais, use_container_width=True)


        st.markdown("---")

        st.subheader("Top 5 Estados com Menos Autos")

        # Destacar Top 5 com Menos Autos
        top5_menos_siglas = autos_por_uf.nsmallest(5, 'Total_Autos')['sigla'].tolist()
        brasil_gdf['destaque_menos'] = brasil_gdf['sigla'].apply(
            lambda sigla: 'Top 5 Menos' if sigla in top5_menos_siglas else 'Outros'
        )

        fig_mapa_menos = px.choropleth(
            brasil_gdf, geojson=brasil_gdf.geometry, locations=brasil_gdf.index,
            color='destaque_menos',
            hover_name='sigla', hover_data={'Total_Autos': True},
            projection='mercator',
            color_discrete_map={'Top 5 Menos': '#31A354', 'Outros': 'lightgrey'}, # Cor verde para menos autos
            title=f'Destaque dos 5 Estados com Menos Autos ({year_range[0]}-{year_range[1]})'
        )
        fig_mapa_menos.update_geos(fitbounds="locations", visible=False)
        fig_mapa_menos.update_layout(margin={"r":0, "t":40, "l":0, "b":0})
        st.plotly_chart(fig_mapa_menos, use_container_width=True)

    except Exception as e:
        st.error(f"Não foi possível gerar os mapas: {e}")


# --- Aba 4: Exportação ---
with tab5:
    st.header("Exportar Dados Filtrados")
    st.write("Use o botão abaixo para fazer o download dos dados que correspondem aos filtros atualmente selecionados na barra lateral.")

    csv_para_download = convert_df_to_csv(df_filtrado)

    st.download_button(
       label="📥 Baixar Dados em CSV",
       data=csv_para_download,
       file_name=f'dados_infracoes_{year_range[0]}_{year_range[1]}.csv',
       mime='text/csv',
    )

Writing app.py


In [ ]:
!curl ipv4.icanhazip.com

34.143.243.91


In [ ]:
!streamlit run app.py &>/content/logs.txt & npx localtunnel --port 8501

⠙⠹your url is: https://shaky-colts-shout.loca.lt
^C
